In [53]:
import os
import sys
sys.path.append('../script')

import getpass
import pandas as pd
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage
import openpyxl
from preprocess import clean_unnamed_header, fill_undefined_sequentially, forward_fill_column_nans
from utils import read_file, get_feature_name_content, format_nested_dict_for_llm
from metadata import get_number_of_row_header, convert_df_headers_to_nested_dict, convert_df_rows_to_nested_dict
from llm import get_schema, get_feature_names, single_agent, splitter
from utils import format_row_dict_for_llm, format_col_dict_for_llm
from extract_df import render_filtered_dataframe, parse_row_paths, parse_col_paths, create_row_condition, create_column_tuples, get_extraction_stats
import json
import dotenv
dotenv.load_dotenv("../script/.env")

True

In [3]:
print("Preprocessing...")
# Initialize the Gemini 2.5 Flash model
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash-preview-04-17",
    google_api_key=os.getenv("GOOGLE_API_KEY_1"),
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

file_path = '../data/real/example2.xlsx'

feature_names = get_feature_names(file_path, llm)
_, feature_name_result = get_feature_name_content(file_path, feature_names)
number_of_row_header = get_number_of_row_header(file_path)
df = pd.read_excel(file_path, header=list(range(0,number_of_row_header)))
df = clean_unnamed_header(df, number_of_row_header)
df = fill_undefined_sequentially(df,feature_name_result["feature_rows"])
df = forward_fill_column_nans(df,feature_name_result["feature_rows"])

row_dict = convert_df_rows_to_nested_dict(df, feature_name_result["feature_rows"])
col_dict = convert_df_headers_to_nested_dict(df, feature_name_result["feature_cols"])

feature_cols = feature_name_result["feature_cols"]
feature_rows = feature_name_result["feature_rows"]
    # Format dictionaries using the new formatting functions
row_structure = format_row_dict_for_llm(row_dict, feature_rows)
col_structure = format_col_dict_for_llm(col_dict)
print(col_structure)
print("=" * 50)
print("Excel Chatbot Ready!")
print("=" * 50)
print("Commands:")
print("- Enter your query in natural language")
print("- Type 'single' to use single agent mode")
print("- Type 'splitter' to use multi-agent splitter mode")
print("- Type 'sanity check' to see data structure")
print("- Type 'exit' or 'quit' to stop")
print("=" * 50)

# Default mode
current_mode = "splitter"
print(f"Current mode: {current_mode}")

Preprocessing...
### Thinking
The provided content has multiple header rows (four rows in total) which define the columns starting from `Năm 2024`. These header rows show a clear hierarchical structure horizontally, representing time periods (Year, Half-year, Quarter, Month, Total). This indicates that these columns are feature columns.
The initial columns (`TT`, `Chỉ tiêu`, `Số liệu`, `Phân loại`, `Loại hình`) show a vertical hierarchical structure in the data rows, where values are nested within the preceding columns (e.g., `Kế hoạch (Tập đoàn)` is under `DOANH THU`, `Sản xuất` is under `Kế hoạch (Tập đoàn)`). This structure defines the rows of the table.
The combination of multiple header rows defining feature columns and hierarchical structure in the initial columns defining the rows is characteristic of a matrix table.
The column `TT` appears to be an index or order column and should be excluded from the feature rows. The columns `Chỉ tiêu`, `Số liệu`, `Phân loại`, and `Loại hình`

In [ ]:

# Get user input
query = "Xét chỉ tiêu liên quan tới vốn, cho tôi biết tổng cộng của 6 tháng đầu và 6 tháng sau, đồng thời tổng cả năm"

if current_mode == "single":
    result = single_agent(query, feature_rows, feature_cols, row_dict, col_dict, llm)
else:  # splitter mode
    result = splitter(query, feature_rows, feature_cols, row_structure, col_structure, llm)

In [57]:
row_paths = parse_row_paths(result['row_selection'])

In [37]:
condition1 = df[('Chỉ tiêu', 'Header', 'Header', 'Header')].isin(['Vốn vay', 'Vốn chủ sở hữu'])
condition2 = df[('Chỉ tiêu', 'Header', 'Header', 'Header')].isin(['Nợ phải trả/ Vốn chủ sở hữu']) & df[('Phân loại', 'Header', 'Header', 'Header')].isin(['Thực hiện'])
condition = condition1 | condition2
a = df[condition]
a

TT                     Chỉ tiêu    Số liệu  Phân loại  Loại hình  \
    Header                       Header     Header     Header     Header   
    Header                       Header     Header     Header     Header   
    Header                       Header     Header     Header     Header   
169   12.2  Nợ phải trả/ Vốn chủ sở hữu  Undefined  Thực hiện  Undefined   
179     16               Vốn chủ sở hữu  Undefined  Undefined  Undefined   
180     17                      Vốn vay  Undefined  Undefined  Undefined   

           Năm 2024                                  ...                   \
    6 Tháng đầu năm                                  ... 6 Tháng cuối năm   
              Quý I                          Quý II  ...          Quý III   
           Tháng 01 Tháng 02 Tháng 03 Cộng Tháng 04  ...         Tháng 07   
169             NaN      NaN      NaN  NaN      NaN  ...              NaN   
179             NaN      NaN      NaN  NaN      NaN  ...              NaN   
180             NaN      NaN      NaN  NaN      NaN  ...              NaN   

                                                                             
                                                                  Tổng cộng  
                             Quý IV                          Cộng    Header  
    Tháng 08 Tháng 09 Cộng Tháng 10 Tháng 11 Tháng 12 Cộng Header    Header  
169      NaN      NaN  NaN      NaN      NaN      NaN  NaN    NaN       NaN  
179      NaN      NaN  NaN      NaN      NaN      NaN  NaN    NaN       NaN  
180      NaN      NaN  NaN      NaN      NaN      NaN  NaN    NaN       NaN  

[3 rows x 24 columns]

In [62]:
import pandas as pd

with pd.ExcelWriter("test.xlsx", engine='openpyxl') as writer:
    a.to_excel(writer, sheet_name='Sheet1', index=True)

In [49]:
a[[(       'TT',           'Header',  'Header',   'Header'),
            ( 'Chỉ tiêu',           'Header',  'Header',   'Header'),
            (  'Số liệu',           'Header',  'Header',   'Header'),
            ('Phân loại',           'Header',  'Header',   'Header'),
            ('Loại hình',           'Header',  'Header',   'Header'),
            ( 'Năm 2024',  '6 Tháng đầu năm',   'Quý I', 'Cộng')
            ]]

,TT,Chỉ tiêu,Số liệu,Phân loại,Loại hình,Năm 2024
,Header,Header,Header,Header,Header,6 Tháng đầu năm
,Header,Header,Header,Header,Header,Quý I
,Header,Header,Header,Header,Header,Cộng
169,12.2,Nợ phải trả/ Vốn chủ sở hữu,Undefined,Thực hiện,Undefined,NaN
179,16,Vốn chủ sở hữu,Undefined,Undefined,Undefined,NaN
180,17,Vốn vay,Undefined,Undefined,Undefined,NaN
